---
title: IO utils
---

## Inspirations

- [FileIO.jl](https://juliaio.github.io/FileIO.jl/stable)
    - [FileIO.jl/src/loadsave.jl at master · JuliaIO/FileIO.jl](https://github.com/JuliaIO/FileIO.jl/blob/master/src/loadsave.jl)
- [Intake](https://intake.readthedocs.io/en/latest/index.html)
    - [intake/intake/readers/datatypes.py at master · intake/intake](https://github.com/intake/intake/blob/master/intake/readers/datatypes.py)
    - [Easy loading and saving · Issue #819 · intake/intake](https://github.com/intake/intake/issues/819)

Reference:

- [7. Input and Output — Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)

In [ ]:
# | default_exp io/__init__
# | export
from intake.readers.datatypes import recommend, JSONFile, PickleFile, Feather2
import importlib
from functools import partial
from pydantic import validate_call
from pathlib import Path

In [ ]:
# | exporti
maps = {
    JSONFile: ["beforerr.io.json"],
    PickleFile: ["beforerr.io.pickle"],
    Feather2: ["beforerr.io.arrow_ipc"],
}


def checkpath(file):
    # Placeholder implementation, replace with actual path checking logic
    pass


def query_datatype(file: str):
    """query the datatype of a file

    See also `os.path.splitext(file)` to get the file extension
    """
    datatypes = recommend(file)
    if JSONFile in datatypes:
        return JSONFile
    if Feather2 in datatypes:
        return Feather2
    else:
        return PickleFile


def applicable_func(datatype, func="load"):
    libraries = maps[datatype]
    lib = libraries[0]
    mod = importlib.import_module(lib)
    return getattr(mod, func)

In [ ]:
# | export
@validate_call
def action(func, file: Path, *args, **kwargs):
    checkpath(file)
    dp = query_datatype(file.as_posix())
    return applicable_func(dp, func)(file, *args, **kwargs)


load = partial(action, "load")
save = partial(action, "save")

## Test

In [ ]:
fp = "test/test.json"
data = load(fp)